In [ ]:
# Plot graphs for each individual behaviors 
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import glob
import re
from matplotlib.backends.backend_pdf import PdfPages

# Specify the path to your CSV files
file_paths = glob.glob("./output_EGD_with_Adam_*200_steps).csv")
# Set file's name to save plots
save_title = "Undefined"
if str(file_paths[0]).__contains__("EGD_with_Adam"):
    save_title = "Exponentiated Gradient Descent"
elif str(file_paths[0]).__contains__("PGD"):
    save_title = "Projected Gradient Descent"

# Initialize dictionaries to store the data
epoch_data = {}
continuous_loss_data = {}
discrete_loss_data = {}

# Regular expression to find the substring between `)_` and `.csv`
pattern = r'\)_([^\.]+)\.csv'

pdf_title = save_title+" (Individual Behaviors).pdf"
# Initialize the PdfPages object
with PdfPages(pdf_title) as pdf:
    # Loop through each file and extract the necessary data
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        epochs = df['epoch']
        continuous_loss = df['continuous_loss']
        # discrete_loss = df['discrete_loss']
        # Extract label from the file_name
        # Search for the pattern in the input string
        match = re.search(pattern, file_path)
        label = file_path
        # Extract the matched substring if the pattern is found
        if match:
            label = match.group(1)

        # for epoch, closs, dloss in zip(epochs, continuous_loss, discrete_loss):
        for epoch, closs in zip(epochs, continuous_loss):
            if epoch not in epoch_data:
                epoch_data[epoch] = []
                continuous_loss_data[epoch] = []
                discrete_loss_data[epoch] = []
            epoch_data[epoch].append(epoch)
            if math.isnan(closs):
                break
            continuous_loss_data[epoch].append(closs)
            # discrete_loss_data[epoch].append(dloss)

        # Plot the results
        plt.figure(figsize=(6, 4))
        plt.plot(epochs, continuous_loss, label='Continuous Loss', color='blue')
        # plt.plot(epochs, discrete_loss, label='Discrete Loss', color='red')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(label[label.find('('):-4])
        plt.legend()
        plt.grid(True)

        # Save the current plot to the PDF file
        pdf.savefig()
        plt.show()
        plt.close()  # Close the current figure to release memory

# Calculate the mean losses for each epoch
epochs = sorted(epoch_data.keys())
median_continuous_loss = [np.median(continuous_loss_data[epoch]) for epoch in epochs]
# median_discrete_loss = [np.median(discrete_loss_data[epoch]) for epoch in epochs]

print("Continuous Loss:", median_continuous_loss)
print("max:", np.max(median_continuous_loss), end=", ")
print("min:", np.min(median_continuous_loss))
# print("Discrete Loss:", median_discrete_loss)
# print("max:", np.max(median_discrete_loss), end=", ")
# print("min:", np.min(median_discrete_loss))


In [ ]:
pdf_title = save_title+" (Median).pdf"
# Plot the results
plt.figure(figsize=(6, 4))
plt.plot(epochs, median_continuous_loss, label='Median Continuous Loss', color='blue')
# plt.plot(epochs, median_discrete_loss, label='Median Discrete Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(label=save_title)
plt.legend()
plt.grid(True)
# fig_name = save_title+".pdf"
plt.savefig(pdf_title, format="pdf", bbox_inches="tight")
plt.show()